In [2]:
import pandas as pd
import numpy as np
import requests
import json

# Import the API key
from config import key 

In [3]:
latitude = 21.3099
longitude = -157.8581
limit = 50
radius = 8000 

city = "Honolulu"
units = "imperial"

#url1 = f"http://api.openweathermap.org/data/2.5/find?lat={latitude}&lon={longitude}&cnt=10&appid={key}&units={units}"
#response = requests.get(url)

#weather_data_list = []

#forecast




In [4]:
url2 = f"http://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={key}&units={units}"

response = requests.get(url2)

data = response.json()

forecast_data_list = []

for item in data["list"]:
    forecast_info = {
        "date": item["dt_txt"].split(" ")[0],
        "temperature": item["main"]["temp"],
        "humidity": item["main"]["humidity"],
        "weather_description": item["weather"][0]["description"],
        "wind_speed": item["wind"]["speed"]
    }
    forecast_data_list.append(forecast_info)

# Converting the list to a DataFrame
forecast_df = pd.DataFrame(forecast_data_list)

daily_summary = forecast_df.groupby('date').agg(
    daily_high=('temperature', 'max'),
    daily_low=('temperature', 'min'),
    avg_humidity=('humidity', 'mean'),
    #weather_descriptions=('weather_description', lambda x: ', '.join(set(x))),
    avg_wind_speed=('wind_speed', 'mean')
).reset_index()


daily_summary.to_json("weather_forecast.json", orient = "records")

In [16]:
cities = ["Honolulu", "Waipahu", "Pearl City", "Kailua", "Kaneohe","Aiea","Ewa Beach","Hale'iwa"]
city_weather = []


for city in cities:
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={key}&units={units}"
    response = requests.get(url)
    if response.status_code == 200:
        weather_data = response.json()
        weather_info = {
            "city": city,
            "coordinates": [weather_data["coord"]["lat"], weather_data["coord"]["lon"]],
            "temperature": weather_data["main"]["temp"],
            "humidity": weather_data["main"]["humidity"],
            "weather": weather_data["weather"][0]["description"],
            "wind_speed": weather_data["wind"]["speed"]
        }
        city_weather.append(weather_info)

weather_df = pd.DataFrame(city_weather)
print(weather_df)

weather_df.to_json("city_weather.json", orient="records")

         city           coordinates  temperature  humidity           weather  \
0    Honolulu  [21.3069, -157.8583]        84.83        53  scattered clouds   
1     Waipahu  [21.3867, -158.0092]        85.42        50         clear sky   
2  Pearl City  [21.3972, -157.9733]        85.71        51  scattered clouds   
3      Kailua  [21.4022, -157.7394]        86.50        53        few clouds   
4     Kaneohe  [21.4181, -157.8036]        86.05        52        few clouds   
5        Aiea  [21.3822, -157.9336]        85.71        52  scattered clouds   
6   Ewa Beach  [21.3156, -158.0072]        85.39        49         clear sky   
7    Hale'iwa  [21.5903, -158.1125]        85.57        47         clear sky   

   wind_speed  
0       12.66  
1       11.50  
2       11.50  
3       12.66  
4       12.66  
5       12.66  
6       11.50  
7       11.50  
